In [75]:
from pathlib import Path

DATASETS = list(Path("../data/err-0.8").rglob("result-experiment*.json"))

In [76]:
for dataset in DATASETS:
    print(dataset)

../data/err-0.8/result-experiment-1.json
../data/err-0.8/result-experiment-2.json


In [77]:
import pandas as pd
import swifter

def _correct_row(row: pd.Series) -> str | None:
    corrected = row["masked"]

    try:
        for fix in row["replace"]:
            corrected = corrected.replace("[MASK]", fix, 1)
    except TypeError:
        return None
    
    return corrected

def correct_row(df: pd.DataFrame) -> pd.DataFrame:
    df["corrected"] = None

    index = df[df["masked"].notnull()].index
    _df = df.loc[index]

    _df["corrected"] = _df.apply(_correct_row, axis=1)
    df.loc[index] = _df
    # df.drop(columns=["replace"], inplace=True)
    return df

In [78]:
from nltk import edit_distance

def calc_edit_distance(row: pd.Series) -> int:
    if row["corrected"] is None:
        return edit_distance(row["sentence"], row["error"])

    return edit_distance(row["sentence"], row["corrected"])

for dataset in DATASETS:
    df = pd.read_json(dataset, orient="records", lines=True)
    df = correct_row(df)
    
    # calculate edit distance 
    df["ED"] = df.swifter.apply(calc_edit_distance, axis=1)
    
    print(df["ED"].mean())

    # print(dataset.name)
    df.to_json(dataset.parent / f"corrected-{dataset.name}", orient="records", lines=True)
    print("correct\t\t", df.iloc[50]["sentence"])
    print("error\t\t", df.iloc[50]["error"])
    print("masked\t\t", df.iloc[50]["masked"])
    print("replace\t\t", df.iloc[50]["replace"])
    print("corrected\t", df.iloc[50]["corrected"])
    print("ED\t\t", df.iloc[50]["ED"])
    print()
    

Pandas Apply: 100%|██████████| 9992/9992 [00:47<00:00, 210.34it/s]


9.638210568454763
correct		 Byl původně vyráběn společností Moto Union Vodňany jako větší sourozenec Dandy 50 .
error		 Byl půvoedně vyráběn společností Moto Uni5on Vodňay jako větší sourozenec Dandy 50  . .
masked		 Byl [MASK] vyráběn společností Moto [MASK] Vodňay jako větší sourozenec Dandy 50 [MASK]
replace		 ['je', 'programu', 'se']
corrected	 Byl je vyráběn společností Moto programu Vodňay jako větší sourozenec Dandy 50 se
ED		 18



Pandas Apply: 100%|██████████| 9992/9992 [00:46<00:00, 213.86it/s]

9.63460768614892
correct		 Byl původně vyráběn společností Moto Union Vodňany jako větší sourozenec Dandy 50 .
error		 Byl půvoedně vyráběn společností Moto Uni5on Vodňay jako větší sourozenec Dandy 50  . .
masked		 Byl [MASK] vyráběn společností Moto [MASK] Vodňay jako větší sourozenec Dandy 50 [MASK]
replace		 ['jeden', 'se', 'Současně']
corrected	 Byl jeden vyráběn společností Moto se Vodňay jako větší sourozenec Dandy 50 Současně
ED		 20

